In [13]:
import os
print(os.environ['PATH'])

C:\Users\Ashvini Alashetty\Anaconda3;C:\Users\Ashvini Alashetty\Anaconda3\Library\mingw-w64\bin;C:\Users\Ashvini Alashetty\Anaconda3\Library\usr\bin;C:\Users\Ashvini Alashetty\Anaconda3\Library\bin;C:\Users\Ashvini Alashetty\Anaconda3\Scripts;C:\Program Files (x86)\Intel\Intel(R) Management Engine Components\iCLS\;C:\Program Files\Intel\Intel(R) Management Engine Components\iCLS\;C:\WINDOWS\system32;C:\WINDOWS;C:\WINDOWS\System32\Wbem;C:\WINDOWS\System32\WindowsPowerShell\v1.0\;C:\WINDOWS\System32\OpenSSH\;C:\Program Files (x86)\Intel\Intel(R) Management Engine Components\DAL;C:\Program Files\Intel\Intel(R) Management Engine Components\DAL;C:\Program Files\MySQL\MySQL Shell 8.0\bin\;C:\Users\Ashvini Alashetty\Anaconda3;C:\Users\Ashvini Alashetty\Anaconda3\Library\mingw-w64\bin;C:\Users\Ashvini Alashetty\Anaconda3\Library\usr\bin;C:\Users\Ashvini Alashetty\Anaconda3\Library\bin;C:\Users\Ashvini Alashetty\Anaconda3\Scripts;C:\Users\Ashvini Alashetty\AppData\Local\Microsoft\WindowsApps;C:

In [1]:
from time import sleep
from random import random
import pandas as pd
import requests
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import urllib.parse as urlparse
from urllib.parse import parse_qs

In [2]:
# Constants
BASE_URL = 'https://www.flipkart.com/'
SEARCH_QUERY = "headphones"
TOP_N_PRODUCTS = 10
REVIEW_PAGES_TO_SCRAPE_FROM_PER_PRODUCT = 100 #10 Reviews exist per page

In [3]:
SAMPLE_URL = "https://www.flipkart.com/boat-rockerz-400-bluetooth-headset/product-reviews/itm14d0416b87d55?pid=ACCEJZXYKSG2T9GS&lid=LSTACCEJZXYKSG2T9GSVY4ZIC&marketplace=FLIPKART&page=1"
r = requests.get(SAMPLE_URL)    
soup = BeautifulSoup(r.content, 'html.parser') 
print(soup.prettify()[:500])

<!DOCTYPE html>
<html lang="en">
 <head>
  <link href="https://rukminim1.flixcart.com" rel="preconnect"/>
  <link href="//static-assets-web.flixcart.com/fk-p-linchpin-web/fk-cp-zion/css/app_modules.chunk.905c37.css" rel="stylesheet"/>
  <link href="//static-assets-web.flixcart.com/fk-p-linchpin-web/fk-cp-zion/css/app.chunk.104e9a.css" rel="stylesheet"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-type"/>
  <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
  <meta content=


In [4]:
rows = soup.find_all('div',attrs={'class':'col _2wzgFH K0kLPL'})
print(f"Count of rows(reviews):{len(rows)}\n\n\n")
# iteration over all blocks
for row in rows:
    # Print a sample row(review html block)
    # print(f"row:\n{row} \n\n")
    
    # finding all rows within the block
    sub_row = row.find_all('div',attrs={'class':'row'})
        
    # extracting text from 1st and 2nd row
    rating = sub_row[0].find('div').text
    print(f"rating:{rating} \n\n")
    
    summary = sub_row[0].find('p').text
    print(f"summary:{summary} \n\n")
    
    review = sub_row[1].find_all('div')[2].text
    print(f"review:{review} \n\n")
    
    location = sub_row[3].find('p',attrs={'class':'_2mcZGG'}).find_all('span')[1].text
    location = "".join(location.split(",")[1:]).strip()
    print(f"location:{location} \n\n")
    
    date = sub_row[3].find_all('p',attrs={'class':'_2sc7ZR'})[1].text
    print(f"date:{date} \n\n")
    
    
    sub_row_2 = row.find_all('div',attrs={'class':'_1e9_Zu'})[0].find_all('span',attrs={'class':'_3c3Px5'})
    
    upvotes = sub_row_2[0].text
    print(f"upvotes:{upvotes} \n\n")
    
    downvotes = sub_row_2[1].text
    print(f"downvotes:{downvotes} \n\n")
    
    break

Count of rows(reviews):10



rating:5 


summary:Worth every penny 


review:It was nice produt. I like it's design a lot.  It's easy to carry. And.   Looked stylish. 


location:Kadirur 


date:Jan, 2020 


upvotes:3832 


downvotes:289 




In [5]:
## Search based Product URL Discovery
def get_popular_product_s_titles_and_urls(search_query : str, popular_products_count_limit : int = None):
    
    search_url = f"{BASE_URL}search?q={search_query}&sort=popularity"
    search_response = requests.get(search_url)
    
    # Pause the loop for 1-3 seconds to simulate natural setting not overwhelm the server with back to back requests without any pause
    # sleep(randint(1,3))
    
    search_html_soup = BeautifulSoup(search_response.content, 'html.parser')
    search_results_products = search_html_soup.find_all('div',attrs={'class':'_4ddWXP'})
    
    product_titles, product_urls = [],[]
    
    product_count = 0
    
    for product in tqdm(search_results_products, desc="Search Results Iteration", position=0, leave=True):
        
        ad_mention_subrow = product.find("div", attrs={"class":"_4HTuuX"})
        
        is_ad = not not ad_mention_subrow
        
        if not is_ad:
            
            title_mention_subrow = product.find("a", attrs={"class":"s1Q9rs"})
            
            product_title = title_mention_subrow["title"]
            product_relative_url = title_mention_subrow["href"]
            product_url = urljoin(BASE_URL,product_relative_url)
            
            parsed_url = urlparse.urlparse(product_url)
            parsed_url_path = parsed_url.path
            parsed_url_path_split = parsed_url_path.split("/")
            parsed_url_path_split[2] = "product-reviews"
            parsed_url_path_modified = "/".join(parsed_url_path_split)
            parsed_url_modified = parsed_url._replace(path=parsed_url_path_modified)
            product_url = parsed_url_modified.geturl()
            
            product_titles.append(product_title)
            product_urls.append(product_url)
            
            product_count += 1
            
            if popular_products_count_limit and (product_count >= popular_products_count_limit):
                break
                
    return product_titles, product_urls

In [6]:
## Collect Product Page URLs for Top 10 Popular Products for 'Headphones' search query
## Specify Search Query and Popular Product Count Limit(optional)
product_titles, product_urls = get_popular_product_s_titles_and_urls(SEARCH_QUERY, TOP_N_PRODUCTS);

Search Results Iteration:   0%|          | 0/40 [00:00<?, ?it/s]

In [7]:
## Iterating over multiple products and multiple pages
from prettytable import PrettyTable
x = PrettyTable()
x.field_names = ["# Products", "# Reviews Per Page", "# Pages", "# Total Reviews Count"]
x.add_row([len(product_urls), 10, REVIEW_PAGES_TO_SCRAPE_FROM_PER_PRODUCT, len(product_urls)*10*REVIEW_PAGES_TO_SCRAPE_FROM_PER_PRODUCT])
print(x)

+------------+--------------------+---------+-----------------------+
| # Products | # Reviews Per Page | # Pages | # Total Reviews Count |
+------------+--------------------+---------+-----------------------+
|     10     |         10         |   100   |         10000         |
+------------+--------------------+---------+-----------------------+


In [9]:
dataset = []

for idx, url in enumerate(tqdm(product_urls, desc='products')):
    # iterating over review pages
    for i in tqdm(range(1,REVIEW_PAGES_TO_SCRAPE_FROM_PER_PRODUCT+1), desc="review pages", position=0, leave=False):
        parsed = urlparse.urlparse(url)
        pid = parse_qs(parsed.query)['pid'][0]
        URL = f"{url}&page={i}"
        
        r = requests.get(URL)
        
        # Pause the loop for 0-1 seconds to simulate natural setting not overwhelm the server with back to back requests without any pause
        sleep(random())
        soup = BeautifulSoup(r.content, 'html.parser') 

        rows = soup.find_all('div',attrs={'class':'col _2wzgFH K0kLPL'})

        for row in rows:

            # finding all rows within the block
            sub_row = row.find_all('div',attrs={'class':'row'})

            # extracting text from 1st 2nd and 4th row
            rating = sub_row[0].find('div').text
            summary = sub_row[0].find('p').text
            summary = summary.strip()
            review = sub_row[1].find_all('div')[2].text
            review = review.strip()
            location=""
            location_row = sub_row[3].find('p',attrs={'class':'_2mcZGG'})
            if location_row:
                location_row = location_row.find_all('span')
                if len(location_row)>=2:
                    location = location_row[1].text
                    location = "".join(location.split(",")[1:]).strip()
            date = sub_row[3].find_all('p',attrs={'class':'_2sc7ZR'})[1].text

            sub_row_2 = row.find_all('div',attrs={'class':'_1e9_Zu'})[0].find_all('span',attrs={'class':'_3c3Px5'})

            upvotes = sub_row_2[0].text
            downvotes = sub_row_2[1].text

            # appending to data
            dataset.append({'product_id':pid, 'product_title':product_titles[idx], 'rating': rating, 'summary': summary, 'review': review, 'location' : location, 'date' : date, 'upvotes' : upvotes, 'downvotes' : downvotes})

products:   0%|          | 0/10 [00:00<?, ?it/s]

review pages:   0%|          | 0/100 [00:00<?, ?it/s]

review pages:   0%|          | 0/100 [00:00<?, ?it/s]

review pages:   0%|          | 0/100 [00:00<?, ?it/s]

review pages:   0%|          | 0/100 [00:00<?, ?it/s]

...

review pages:   0%|          | 0/100 [00:00<?, ?it/s]

review pages:   0%|          | 0/100 [00:00<?, ?it/s]

review pages:   0%|          | 0/100 [00:00<?, ?it/s]

review pages:   0%|          | 0/100 [00:00<?, ?it/s]

review pages:   0%|          | 0/100 [00:00<?, ?it/s]

In [10]:
## View Sample set of reviews that we collected
df = pd.DataFrame(dataset)

with pd.option_context('display.max_colwidth', -1):
    display(df.head(5))
    display(df.tail(5))

,product_id,product_title,rating,summary,review,location,date,upvotes,downvotes
0,ACCGC4GN5SQR7ZCZ,"OnePlus Bullets Wireless Z2 with Fast Charge, 30 Hrs Battery Life, Earphones with mic Bluetooth Headset",5,Simply awesome,"Writing this review after using this product for two weeks , so u can find ur neck band ,Prose :-* A loude and clear music* Amazing Bass quality , 12.4mm driv just blows up mind clear Instrument sounds* Good batry back up * Nice build quality, and hand free usage design of buds is good fits in ear , * Clear voice calls Cons ;- * Heavy bass sometimes suppress vocals * Absence of Quick switch option ☹️* BLUTOOTH V5.OOver All Like me other bass or loud music lovers , Don't think jus...",Dharwad,5 months ago,410,88
1,ACCGC4GN5SQR7ZCZ,"OnePlus Bullets Wireless Z2 with Fast Charge, 30 Hrs Battery Life, Earphones with mic Bluetooth Headset",4,Nice product,"👉Sound quality is excellent, Bass is awesome, balanced bass and treble,👉Build quality is good but One Plus Z Bass Edition has better build quality in comparison of this,👉Battery backup is awesome, as per commitment it is actual approximately 28 hours,👉Wire texture looks premium quality, very comfortable,👉Overall value for money neckband wireless earphone in this price segment, don't go with bad reviews, its really awesome..",Varanasi,5 months ago,2397,610
2,ACCGC4GN5SQR7ZCZ,"OnePlus Bullets Wireless Z2 with Fast Charge, 30 Hrs Battery Life, Earphones with mic Bluetooth Headset",5,Simply awesome,Excellent product. Superb bass. Worth for money. Full clear and crisp sound. Battery is very powerful.,Bengaluru,6 months ago,1615,439
3,ACCGC4GN5SQR7ZCZ,"OnePlus Bullets Wireless Z2 with Fast Charge, 30 Hrs Battery Life, Earphones with mic Bluetooth Headset",5,Simply awesome,Superb product actually I won’t prefer wireless headphones but i loved this one ❤️.. base i nice not too much high and vocals are superb just go for it,Bengaluru,6 months ago,748,195
4,ACCGC4GN5SQR7ZCZ,"OnePlus Bullets Wireless Z2 with Fast Charge, 30 Hrs Battery Life, Earphones with mic Bluetooth Headset",5,Mind-blowing purchase,"This is worth every penny you spend. I doesn't disappoint in any way. Delivery was 2 days and charged it before using and I have been using it for 10 hours straight and its still 90% charge. I don't know currently how much time it would last, though I won't count till then because I trust the brand and so far battery looks awesome.The sound quality is different from the boat rockerz I was using as it consists more mettalic type of bass and it feels amazing. Its easy to use, by just seperatin...",Ramanagara,6 months ago,681,194


,product_id,product_title,rating,summary,review,location,date,upvotes,downvotes
2435,ACCFU3G2HEJYHKSS,DAYNEO TwsL21 Mini Wireless Bluetooth Headset / Earphone Bluetooth Headset,5,Perfect product!,Excellent,,6 days ago,0,0
2436,ACCFU3G2HEJYHKSS,DAYNEO TwsL21 Mini Wireless Bluetooth Headset / Earphone Bluetooth Headset,4,Very Good,Good,,6 days ago,0,0
2437,ACCFU3G2HEJYHKSS,DAYNEO TwsL21 Mini Wireless Bluetooth Headset / Earphone Bluetooth Headset,5,Wonderful,Excellent product and it's performance is very nice...i loved it very much,,6 days ago,0,0
2438,ACCFU3G2HEJYHKSS,DAYNEO TwsL21 Mini Wireless Bluetooth Headset / Earphone Bluetooth Headset,4,Nice product,Nice product...,Sibsagar,6 days ago,0,0
2439,ACCGH4KENHHGJAZJ,"BHAVISHU Bluetooth Earphone with Mic HEADPHONE Bluetooth Headset (White, In the Ear)7 Bluetooth Headset",5,Awesome,Product is i7s value for money,,9 days ago,4,1


In [11]:
count_reviews = df.shape[0]
print(f"Count of reviews:{count_reviews}")

Count of reviews:2440


In [12]:
df.to_csv("./flipkart_reviews_dataset.csv", index=False)